In [ ]:
import json
import torch
from torchvision import models, transforms
from PIL import Image
import urllib.request
import numpy as np
import sagemaker
import boto3

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = "aipi510-ml-deployment-bucket"
print("Using Bucket: " + bucket)

Using Bucket: aipi510-ml-deployment-bucket
{'ResponseMetadata': {'RequestId': '2D7BXM23Y8496793', 'HostId': 'XXFrJVLOnSIMBemhEtIS2v38jXcYJ4BbwpqGLzJvD8TESF/3vbHbBm+zRfJEXz14zKayl7KBUjk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'XXFrJVLOnSIMBemhEtIS2v38jXcYJ4BbwpqGLzJvD8TESF/3vbHbBm+zRfJEXz14zKayl7KBUjk=', 'x-amz-request-id': '2D7BXM23Y8496793', 'date': 'Thu, 31 Oct 2024 23:14:21 GMT', 'x-amz-bucket-region': 'us-east-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Name': 'aipi510-ml-deployment-bucket', 'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 0}
Bucket is empty or inaccessible.


In [2]:
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
urllib.request.urlretrieve(url, "imagenet_class_index.json")

with open("imagenet_class_index.json") as f:
    labels = json.load(f)

In [ ]:
# Load the ResNet-50 model pre-trained on ImageNet
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

In [4]:
img_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
img = Image.open("./img/Efes.png")
img_tensor = img_transform(img).unsqueeze(0)
with torch.no_grad():
    output = model(img_tensor)

probabilities = torch.nn.functional.softmax(output[0], dim=0)
_, predicted_class = probabilities.max(0)
print(labels[predicted_class.item()])

Australian Kelpie
